# Divide text into chunks

We will take the extracted text from the [previous step](./1_2_ExtractTextFomPDFsGemini.ipynb) and divide it into smaller chunks, each focused on a single topic. This approach allows us to create questions that can be answered using just one chunk of text.

## Gemini

### Create the system prompt

Start by creating the system prompt to teach the LLM how to divide the text into chunks. Feel free to modify the prompt as you like.

In [1]:
system_prompt = """
## Task description

You are an expert at dividing document text into coherent chunks, where each chunk focuses on a single topic. 
Chunks should be semantically complete — each must make sense on its own and not cut off sentences or important context. 
The goal is to create topic-focused text segments that can later be used to generate precise, self-contained questions.

You do not need to make the chunks small or fixed in size. 
Instead, focus on grouping sentences that belong together logically — for example, a section describing one feature, a single FAQ group, or a subchapter of a manual.

### Important instructions:
- Do not paraphrase, summarize, or omit any content.
- Do not drop any text. All extracted text must appear in some chunk.
- Keep descriptions of figures, tables, or diagrams together with their corresponding text.
- Ensure each chunk is self-contained and thematically focused.
- Maintain the original formatting (lists, bullet points, bold text, etc.) as much as possible.
- Output only the JSON array. No explanations or text outside of JSON.

Dividing text into focused chunks helps create high-quality datasets for evaluating embedding models. 
Your expertise in this task will help ensure strong, semantically consistent results.

## Output format

Return your result as a JSON array of objects in this structure:
[
    {
    "text_chunk": "The text chunk focusing on a single topic."
    },
    ...
]

## Examples

### Example 1

- Text:

Task view
Quickly switch between opened apps and programs using the Task view feature, you can also use Task view to switch between desktops.

An image shows the Windows 11 Task View, displaying thumbnails of open application windows (Microsoft Edge, Microsoft Store, File Explorer) above the taskbar. It also shows options for "Desktop 1" and "New desktop".

Launching Task view
-   Position your mouse pointer over the Task View icon on the taskbar and click it.
-   Press Windows key + Tab on your keyboard.

Widgets
You can launch Widgets for quick access of important info.

An image shows the Windows 11 desktop with the Widgets panel open on the left side. The panel displays weather, stock market information, top news stories, and sports scores.

Launching Widgets
-   Position your mouse pointer over the Widgets icon on the taskbar and click it.
-   Press Windows key + W on your keyboard.

Customizing Widgets
An image of the Widgets panel is shown, with a callout highlighting the "more options" (three dots) icon at the top right.
-   Click/Tap the "..." icon, then select **Manage interests** to change the content that shows up in the Widgets.

Snap feature
The Snap feature displays apps side-by-side, allowing you to work or switch between apps.

An image shows four applications (Calendar, Microsoft Store, Microsoft Edge, File Explorer) arranged on the screen using the Snap feature, filling the entire display.

Snap hotspots
You can drag and drop apps to these hotspots to snap them into place.

An image of the Windows 11 desktop. The screen is overlaid with transparent circles indicating the different "snap hotspots" where a window can be dragged to automatically resize and position it (e.g., left half, right half, to half, corners).

Using Snap
1.  Launch the app you wish to snap.
2.  Drag the title bar of your app and drop the app to the edge of the screen to snap.
3.  Launch another app and repeat the above steps to snap another app.

1.  Launch the app you wish to snap.
2.  Press and hold the Windows key, then use the arrow keys to snap the app.
3.  Launch another app and repeat the above steps to snap another app.

- Output:

[
    {
        "text_chunk": "Task view\nQuickly switch between opened apps and programs using the Task view feature, you can also use Task view to switch between desktops.\n\nAn image shows the Windows 11 Task View, displaying thumbnails of open application windows (Microsoft Edge, Microsoft Store, File Explorer) above the taskbar. It also shows options for \"Desktop 1\" and \"New desktop\".\n\nLaunching Task view\n- Position your mouse pointer over the Task View icon on the taskbar and click it.\n- Press Windows key + Tab on your keyboard."
    },
    {
        "text_chunk": "Widgets\nYou can launch Widgets for quick access of important info.\n\nAn image shows the Windows 11 desktop with the Widgets panel open on the left side. The panel displays weather, stock market information, top news stories, and sports scores.\n\nLaunching Widgets\n- Position your mouse pointer over the Widgets icon on the taskbar and click it.\n- Press Windows key + W on your keyboard.\n\nCustomizing Widgets\nAn image of the Widgets panel is shown, with a callout highlighting the \"more options\" (three dots) icon at the top right.\n- Click/Tap the \"...\" icon, then select Manage interests to change the content that shows up in the Widgets."
    },
    {
        "text_chunk": "Snap feature\nThe Snap feature displays apps side-by-side, allowing you to work or switch between apps.\n\nAn image shows four applications (Calendar, Microsoft Store, Microsoft Edge, File Explorer) arranged on the screen using the Snap feature, filling the entire display.\n\nSnap hotspots\nYou can drag and drop apps to these hotspots to snap them into place.\n\nAn image of the Windows 11 desktop. The screen is overlaid with transparent circles indicating the different \"snap hotspots\" where a window can be dragged to automatically resize and position it (e.g., left half, right half, to half, corners).\n\nUsing Snap\n1. Launch the app you wish to snap.\n2. Drag the title bar of your app and drop the app to the edge of the screen to snap.\n3. Launch another app and repeat the above steps to snap another app.\n\n1. Launch the app you wish to snap.\n2. Press and hold the Windows key, then use the arrow keys to snap the app.\n3. Launch another app and repeat the above steps to snap another app."
    },
]

### Example 2

- Text:

Tips and FAQs

Useful tips for your Notebook PC
To help you maximize the use of your Notebook PC, maintain its system performance, and ensure all your data are kept secured, here are some useful tips that you can follow:
*   Update Windows periodically to ensure that your applications have the latest security settings.
*   Update MyASUS to ensure that you have the latest settings for ASUS exclusive applications, drivers, and utilities.
*   Use an anti-virus software to protect your data and keep this updated too.
*   Unless absolutely necessary, refrain from using force shutdown to turn off your Notebook PC.
*   Always backup your data and make it a point to create a backup data in an external storage drive.
*   If you are not using your Notebook PC for a long period of time, ensure to charge the battery power to 50% then power off your Notebook PC and disconnect the AC power adapter.
*   If you are constantly using AC power for your Notebook PC, set the Battery Health Charging to Balanced mode in MyASUS.
*   Disconnect all external devices and ensure you have the following items prior to resetting your Notebook PC:
    -   Product key for your operating systems and other installed applications
    -   Backup data
    -   Log in ID and password
    -   Internet connection information
*   Visit our support site for troubleshooting and view some of the most frequently asked questions at https://www.asus.com/support.

Hardware FAQs
1.  **A black dot, or sometimes a colored dot, appears onscreen when I turn on the Notebook PC. What should I do?**
    Although these dots normally appear onscreen, they will not affect your system. If the incident continues and subsequently affects system performance, consult an authorized ASUS service center.
2.  **My display panel has an uneven color and brightness. How can I fix this?**
    The color and brightness of your display panel may be affected by the angle and current position of your Notebook PC. The brightness and color tone of your Notebook PC may also vary per model. You may use the function keys or the display settings in your operating system to adjust the appearance of your display panel.
3.  **How can I maximize my Notebook PC ‘s battery life?**
    You can try doing any of the following suggestions:
    *   Use the function keys to adjust the display brightness.
    *   If you are not using any Wi-Fi connection, switch your system into **Airplane mode**.
    *   Disconnect unused USB devices.
    *   Close unused applications, especially those that take up too much system memory.
4.  **My battery charge indicator does not light up. What’s wrong?**
    *   Check whether the power adapter or battery pack is attached correctly. You may also disconnect the power adapter or battery pack, wait for a minute, then reconnect them again to the power outlet and Notebook PC.
    *   If the problem still exists, contact your local ASUS service center for assistance.
5.  **Why is my touchpad not working?**
    Press FN + F10 to enable your touchpad.
6.  **When I play audio and video files, why can’t I hear any sound from my Notebook PC’s audio speakers?**
    You can try doing any of the following suggestions:
    *   Press M2 to turn up the speaker volume.
    *   Check if your speakers were set to mute.
    *   Check if a headphone jack is connected to your Notebook PC and remove it.
7.  **What should I do if my Notebook PC’s power adapter gets lost or my battery stops working?**
    Contact your local ASUS service center for assistance.
8.  **My Notebook PC cannot do keystrokes correctly because my cursor keeps on moving. What should I do?**
    Make sure that nothing accidentally touches or presses on your touchpad while you type on the keyboard. You can also disable your touchpad by pressing FN + F10.
9.  **I have disabled sleep mode, but why does my Notebook PC still go into sleep mode when I close the lid?**
    Closing the lid when the system is under high load forces your Notebook PC to go into sleep mode to prevent overheating.

Software FAQs
1.  **When I turn on my Notebook PC, the power indicator lights up but nothing appears on my screen. What can I do to fix this?**
    You can try doing any of the following suggestions:
    *   Force shutdown your Notebook PC by pressing the power button for at least ten (10) seconds. Check if the power adapter and battery pack are inserted correctly then turn on your Notebook PC.
    *   If the problem still exists, contact your local ASUS service center for assistance.
2.  **What should I do when my screen displays this message: “Remove disks or other media. Press any key to restart.”?**
    You can try doing any of the following suggestions:
    *   Remove all connected USB devices then restart your Notebook PC.
    *   Remove any optical discs that are left inside the optical drive then restart.
    *   If the problem still exists, your Notebook PC might have a memory storage problem. Contact your local ASUS service center for assistance.
3.  **My Notebook PC boots slower than usual and my operating system lags. How can I fix this?**
    Delete the applications you recently installed or were not included with your operating system package then restart your system.
4.  **My Notebook PC does not boot up. How can I fix this?**
    You can try doing any of the following suggestions:
    *   Remove all connected devices to your Notebook PC then restart your system.
    *   If the problem still exists, contact your local ASUS service center for assistance.
5.  **Why can’t my Notebook PC wake up from sleep mode?**
    *   You need to press the power button to resume your last working state.
    *   Your system might have completely used up its battery power. Connect the power adapter to your Notebook PC and connect to a power outlet then press the power button.

- Output:

[
    {
        "text_chunk": "Tips and FAQs\n\nUseful tips for your Notebook PC\nTo help you maximize the use of your Notebook PC, maintain its system performance, and ensure all your data are kept secured, here are some useful tips that you can follow:\n* Update Windows periodically to ensure that your applications have the latest security settings.\n* Update MyASUS to ensure that you have the latest settings for ASUS exclusive applications, drivers, and utilities.\n* Use an anti-virus software to protect your data and keep this updated too.\n* Unless absolutely necessary, refrain from using force shutdown to turn off your Notebook PC.\n* Always backup your data and make it a point to create a backup data in an external storage drive.\n* If you are not using your Notebook PC for a long period of time, ensure to charge the battery power to 50% then power off your Notebook PC and disconnect the AC power adapter.\n* If you are constantly using AC power for your Notebook PC, set the Battery Health Charging to Balanced mode in MyASUS.\n* Disconnect all external devices and ensure you have the following items prior to resetting your Notebook PC:\n    -   Product key for your operating systems and other installed applications\n    -   Backup data\n    -   Log in ID and password\n    -   Internet connection information\n* Visit our support site for troubleshooting and view some of the most frequently asked questions at https://www.asus.com/support."
    },
    {
        "text_chunk": "Hardware FAQs\n1.  **A black dot, or sometimes a colored dot, appears onscreen when I turn on the Notebook PC. What should I do?**\n    Although these dots normally appear onscreen, they will not affect your system. If the incident continues and subsequently affects system performance, consult an authorized ASUS service center.\n2.  **My display panel has an uneven color and brightness. How can I fix this?**\n    The color and brightness of your display panel may be affected by the angle and current position of your Notebook PC. The brightness and color tone of your Notebook PC may also vary per model. You may use the function keys or the display settings in your operating system to adjust the appearance of your display panel.\n3.  **How can I maximize my Notebook PC ‘s battery life?**\n    You can try doing any of the following suggestions:\n    * Use the function keys to adjust the display brightness.\n    * If you are not using any Wi-Fi connection, switch your system into **Airplane mode**.\n    * Disconnect unused USB devices.\n    * Close unused applications, especially those that take up too much system memory.\n4.  **My battery charge indicator does not light up. What’s wrong?**\n    * Check whether the power adapter or battery pack is attached correctly. You may also disconnect the power adapter or battery pack, wait for a minute, then reconnect them again to the power outlet and Notebook PC.\n    * If the problem still exists, contact your local ASUS service center for assistance.\n5.  **Why is my touchpad not working?**\n    Press FN + F10 to enable your touchpad.\n6.  **When I play audio and video files, why can’t I hear any sound from my Notebook PC’s audio speakers?**\n    You can try doing any of the following suggestions:\n    * Press M2 to turn up the speaker volume.\n    * Check if your speakers were set to mute.\n    * Check if a headphone jack is connected to your Notebook PC and remove it.\n7.  **What should I do if my Notebook PC’s power adapter gets lost or my battery stops working?**\n    Contact your local ASUS service center for assistance.\n8.  **My Notebook PC cannot do keystrokes correctly because my cursor keeps on moving. What should I do?**\n    Make sure that nothing accidentally touches or presses on your touchpad while you type on the keyboard. You can also disable your touchpad by pressing FN + F10.\n9.  **I have disabled sleep mode, but why does my Notebook PC still go into sleep mode when I close the lid?**\n    Closing the lid when the system is under high load forces your Notebook PC to go into sleep mode to prevent overheating."
    },
    {
        "text_chunk": "Software FAQs\n1.  **When I turn on my Notebook PC, the power indicator lights up but nothing appears on my screen. What can I do to fix this?**\n    You can try doing any of the following suggestions:\n    * Force shutdown your Notebook PC by pressing the power button for at least ten (10) seconds. Check if the power adapter and battery pack are inserted correctly then turn on your Notebook PC.\n    * If the problem still exists, contact your local ASUS service center for assistance.\n2.  **What should I do when my screen displays this message: “Remove disks or other media. Press any key to restart.”?**\n    You can try doing any of the following suggestions:\n    * Remove all connected USB devices then restart your Notebook PC.\n    * Remove any optical discs that are left inside the optical drive then restart.\n    * If the problem still exists, your Notebook PC might have a memory storage problem. Contact your local ASUS service center for assistance.\n3.  **My Notebook PC boots slower than usual and my operating system lags. How can I fix this?**\n    Delete the applications you recently installed or were not included with your operating system package then restart your system.\n4.  **My Notebook PC does not boot up. How can I fix this?**\n    You can try doing any of the following suggestions:\n    * Remove all connected devices to your Notebook PC then restart your system.\n    * If the problem still exists, contact your local ASUS service center for assistance.\n5.  **Why can’t my Notebook PC wake up from sleep mode?**\n    * You need to press the power button to resume your last working state.\n    * Your system might have completely used up its battery power. Connect the power adapter to your Notebook PC and connect to a power outlet then press the power button."
    }
]

### Example 3

- Text:

Chapter 1:
Hardware Setup

Getting to know your Notebook PC
Front View

NOTE: The keyboard’s layout may vary per region or country. The front view may also vary in appearance depending on the Notebook PC model.

A diagram shows the front view of an open ROG Strix gaming laptop. It is a line drawing with several parts of the laptop numbered with circular callouts.

1.  **Array microphones**
    The array microphones feature echo canceling, noise suppression, and beam forming functions for better voice recognition and audio recording.

2.  **Camera indicator**
    The camera indicator lights up when the built-in camera is in use.

3.  **Camera**
    The built-in camera allows you to take pictures or record videos using your Notebook PC.

4.  **Display panel**
    This high-definition display panel provides excellent viewing features for photos, videos, and other multimedia files on your Notebook PC.

5.  **Status indicators**
    The status indicators help identify the current hardware status of your Notebook PC.

    **Power indicator**
    The power indicator lights up when the Notebook PC is turned on and blinks slowly when the Notebook PC is in sleep mode.

    **Two-color battery charge indicator**
    The two-color LED provides a visual indication of the battery’s charge status. Refer to the following table for details:

Description of a table about the Two-color battery charge indicator. The table has two columns: "Color" and "Status".
*   **Color:** Solid White
    **Status:** The Notebook PC is plugged to a power source and the battery power is 100%.
*   **Color:** Solid Orange
    **Status:** The Notebook PC is plugged to a power source, charging its battery, and the battery power is between 0% and 99%.
*   **Color:** Blinking Orange
    **Status:** The Notebook PC is running on battery mode and the battery power is less than 10%.
*   **Color:** Lights off
    **Status:** The Notebook PC is running on battery mode and the battery power is between 10% to 100%.

**Drive activity indicator**
This indicator lights up when your Notebook PC is accessing the internal storage drives.

6.  **Capital lock indicator**
    This indicator lights up when the capital lock function is activated. Using capital lock allows you to type capitalized letters (such as A, B, C) using your Notebook PC’s keyboard.

7.  **Power button**
    Press the power button to turn your Notebook PC on or off. You can also use the power button to put your Notebook PC to sleep or hibernate mode and wake it up from sleep or hibernate mode.
    In the event that your Notebook PC becomes unresponsive, press and hold the power button for at least ten (10) seconds until your Notebook PC shuts down.

8.  **Keyboard**
    The keyboard provides full-sized QWERTY keys with a comfortable travel depth for typing. It also enables you to use the function keys, allows quick access to Windows, and controls other multimedia functions.
    NOTE: The keyboard layout differs by model or territory.

9.  **Touchpad/NumberPad (on selected models)**
    This allows you to toggle between touchpad and NumberPad.
    The touchpad allows the use of multi-gestures to navigate onscreen, providing an intuitive user experience. It also simulates the functions of a regular mouse.
    NOTE: For more details, refer to the *Using the touchpad* section in this manual.
    The NumberPad allows the use of numeric input.
    NOTE: For more details, refer to the *Using the NumberPad* section in this manual.

Bottom View
NOTE: The bottom view may vary in appearance depending on model.

WARNING! Your Notebook PC can get warm to hot while in use or while charging the battery pack. Do not leave your Notebook PC on your lap or near any part of your body to prevent injury from heat. When working on your Notebook PC, do not place it on surfaces that can block the vents.

IMPORTANT!
*   Battery time varies by usage and by the specifications for this Notebook PC. The battery pack cannot be disassembled.
*   Purchase components from authorized retailers to ensure maximum compatibility and reliability. Consult an ASUS service center or authorized retailers for product service or assistance on proper disassembly or assembly of the product.

An image showing a diagram of the bottom of the laptop. The design is stylized with diagonal vents and geometric patterns. There are callouts pointing to different parts.

1.  **Air vents**
    The air vents allow cool air to enter and warm air to exit the Notebook PC.
    WARNING! Ensure that paper, books, clothing, cables, or other objects do not block any of the air vents or else overheating may occur.

2.  **Audio speakers**
    The built-in audio speakers allow you to hear audio straight from your Notebook PC. Audio features are software-controlled.

3.  **Bottom cover**
    This cover protects the reserved spaces in your Notebook PC for the removable SSD (M.2 2280 slot) and RAM (Random Access Memory) modules.
    NOTE:
    *   For more details, refer to the Service Manual on our multi-language website at https://www.asus.com/support/.
    *   The M.2 2280 slot does not support M.2 SATA SSD.

Right View

An image showing a diagram of the right side of the laptop. There are two callouts.

1.  **USB 3.2 Gen 2 port**
    The USB 3.2 (Universal Serial Bus 3.2) Gen 2 port provides a transfer rate of up to 10 Gbit/s and is backward compatible to USB 2.0.

2.  **Air vents**
    The air vents allow cool air to enter and warm air to exit the Notebook PC.
    WARNING! Ensure that paper, books, clothing, cables, or other objects do not block any of the air vents or else overheating may occur.

Left View

An image showing a diagram of the left side of the laptop, displaying a row of ports and vents with seven callouts.

1.  **Air vents**
    The air vents allow cool air to enter and warm air to exit the Notebook PC.
    WARNING! Ensure that paper, books, clothing, cables, or other objects do not block any of the air vents or else overheating may occur.

2.  **Power (DC) input port** 
    Insert the bundled power adapter into this port to charge the battery pack and supply power to your Notebook PC.
    WARNING! The adapter may become warm to hot while in use. Do not cover the adapter and keep it away from your body while it is connected to a power source.
    IMPORTANT! Use only the bundled power adapter to charge the battery pack and supply power to your Notebook PC.

3.  **LAN port**
    Insert the network cable into this port to connect to a local area network.
    WARNING! This LAN port does not support PoE (Power over Ethernet). Connecting a PoE device to your Notebook PC may result in damage to the system board.

4.  **HDMI output port**
    This port allows you to send audio and video signals from your Notebook PC to an HDMI-compatible device using an HDMI cable.

5.  **Thunderbolt™ 4 port**
    The Thunderbolt™ 4 port is compatible with USB Type-C® standards, and provides a transfer rate of up to 40 Gbit/s for Thunderbolt™ 4 devices. Connect your Notebook PC to a DisplayPort, VGA, DVI, or HDMI external display and experience high-resolution display features via Intel® Thunderbolt™ technology. 
    NOTE: Transfer rate of this port may vary per model.

6.  **USB 3.2 Gen 2 Type-C®/DisplayPort/Power Delivery combo port**
    The USB 3.2 (Universal Serial Bus 3.2) Gen 2 Type-C® port provides a transfer rate of up to 10 Gbit/s and is backward compatible to USB 2.0. Use a USB Type-C® adapter to connect your Notebook PC to an external display. Connect a power source rated 20V/5A to charge the battery pack and supply power to your Notebook PC.
    NOTE: Transfer rate of this port may vary per model.
    WARNING! The adapter may become warm to hot while in use. Do not cover the adapter and keep it away from your body while it is connected to a power source.
    IMPORTANT!
    *   It may take longer to fully charge your Notebook PC through a low-voltage power source.
    *   To prevent any damage, use only power sources rated 20V/5A to charge your Notebook PC with the USB Power Delivery combo port. For more information, consult an ASUS service center for assistance.

7.  **Headphone/Headset/Microphone jack**
    This port allows you to connect amplified speakers or headphones. You can also use this port to connect your headset or an external microphone.

- Output:

[
    {
        "text_chunk": "Getting to know your Notebook PC\nFront View\n\nNOTE: The keyboard’s layout may vary per region or country. The front view may also vary in appearance depending on the Notebook PC model.\n\nA diagram shows the front view of an open ROG Strix gaming laptop. It is a line drawing with several parts of the laptop numbered with circular callouts.\n\n1.  **Array microphones**\n    The array microphones feature echo canceling, noise suppression, and beam forming functions for better voice recognition and audio recording.\n\n2.  **Camera indicator**\n    The camera indicator lights up when the built-in camera is in use.\n\n3.  **Camera**\n    The built-in camera allows you to take pictures or record videos using your Notebook PC.\n\n4.  **Display panel**\n    This high-definition display panel provides excellent viewing features for photos, videos, and other multimedia files on your Notebook PC.\n\n5.  **Status indicators**\n    The status indicators help identify the current hardware status of your Notebook PC.\n\n    **Power indicator**\n    The power indicator lights up when the Notebook PC is turned on and blinks slowly when the Notebook PC is in sleep mode.\n\n    **Two-color battery charge indicator**\n    The two-color LED provides a visual indication of the battery’s charge status. Refer to the following table for details:\n\nDescription of a table about the Two-color battery charge indicator. The table has two columns: \"Color\" and \"Status\".\n* **Color:** Solid White\n    **Status:** The Notebook PC is plugged to a power source and the battery power is 100%.\n* **Color:** Solid Orange\n    **Status:** The Notebook PC is plugged to a power source, charging its battery, and the battery power is between 0% and 99%.\n* **Color:** Blinking Orange\n    **Status:** The Notebook PC is running on battery mode and the battery power is less than 10%.\n* **Color:** Lights off\n    **Status:** The Notebook PC is running on battery mode and the battery power is between 10% to 100%.\n\n**Drive activity indicator**\nThis indicator lights up when your Notebook PC is accessing the internal storage drives.\n\n6.  **Capital lock indicator**\n    This indicator lights up when the capital lock function is activated. Using capital lock allows you to type capitalized letters (such as A, B, C) using your Notebook PC’s keyboard.\n\n7.  **Power button**\n    Press the power button to turn your Notebook PC on or off. You can also use the power button to put your Notebook PC to sleep or hibernate mode and wake it up from sleep or hibernate mode.\n    In the event that your Notebook PC becomes unresponsive, press and hold the power button for at least ten (10) seconds until your Notebook PC shuts down.\n\n8.  **Keyboard**\n    The keyboard provides full-sized QWERTY keys with a comfortable travel depth for typing. It also enables you to use the function keys, allows quick access to Windows, and controls other multimedia functions.\n    NOTE: The keyboard layout differs by model or territory.\n\n9.  **Touchpad/NumberPad (on selected models)**\n    This allows you to toggle between touchpad and NumberPad.\n    The touchpad allows the use of multi-gestures to navigate onscreen, providing an intuitive user experience. It also simulates the functions of a regular mouse.\n    NOTE: For more details, refer to the *Using the touchpad* section in this manual.\n    The NumberPad allows the use of numeric input.\n    NOTE: For more details, refer to the *Using the NumberPad* section in this manual."
    },
    {
        "text_chunk": "Bottom View\nNOTE: The bottom view may vary in appearance depending on model.\n\nWARNING! Your Notebook PC can get warm to hot while in use or while charging the battery pack. Do not leave your Notebook PC on your lap or near any part of your body to prevent injury from heat. When working on your Notebook PC, do not place it on surfaces that can block the vents.\n\nIMPORTANT!\n* Battery time varies by usage and by the specifications for this Notebook PC. The battery pack cannot be disassembled.\n* Purchase components from authorized retailers to ensure maximum compatibility and reliability. Consult an ASUS service center or authorized retailers for product service or assistance on proper disassembly or assembly of the product.\n\nAn image showing a diagram of the bottom of the laptop. The design is stylized with diagonal vents and geometric patterns. There are callouts pointing to different parts.\n\n1.  **Air vents**\n    The air vents allow cool air to enter and warm air to exit the Notebook PC.\n    WARNING! Ensure that paper, books, clothing, cables, or other objects do not block any of the air vents or else overheating may occur.\n\n2.  **Audio speakers**\n    The built-in audio speakers allow you to hear audio straight from your Notebook PC. Audio features are software-controlled.\n\n3.  **Bottom cover**\n    This cover protects the reserved spaces in your Notebook PC for the removable SSD (M.2 2280 slot) and RAM (Random Access Memory) modules.\n    NOTE:\n    * For more details, refer to the Service Manual on our multi-language website at https://www.asus.com/support/.\n    * The M.2 2280 slot does not support M.2 SATA SSD."
    },
    {
        "text_chunk": "Right View\n\nAn image showing a diagram of the right side of the laptop. There are two callouts.\n\n1.  **USB 3.2 Gen 2 port**\n    The USB 3.2 (Universal Serial Bus 3.2) Gen 2 port provides a transfer rate of up to 10 Gbit/s and is backward compatible to USB 2.0.\n\n2.  **Air vents**\n    The air vents allow cool air to enter and warm air to exit the Notebook PC.\n    WARNING! Ensure that paper, books, clothing, cables, or other objects do not block any of the air vents or else overheating may occur."
    },
    {
        "text_chunk": "Left View\n\nAn image showing a diagram of the left side of the laptop, displaying a row of ports and vents with seven callouts.\n\n1.  **Air vents**\n    The air vents allow cool air to enter and warm air to exit the Notebook PC.\n    WARNING! Ensure that paper, books, clothing, cables, or other objects do not block any of the air vents or else overheating may occur.\n\n2.  **Power (DC) input port** \n    Insert the bundled power adapter into this port to charge the battery pack and supply power to your Notebook PC.\n    WARNING! The adapter may become warm to hot while in use. Do not cover the adapter and keep it away from your body while it is connected to a power source.\n    IMPORTANT! Use only the bundled power adapter to charge the battery pack and supply power to your Notebook PC.\n\n3.  **LAN port**\n    Insert the network cable into this port to connect to a local area network.\n    WARNING! This LAN port does not support PoE (Power over Ethernet). Connecting a PoE device to your Notebook PC may result in damage to the system board.\n\n4.  **HDMI output port**\n    This port allows you to send audio and video signals from your Notebook PC to an HDMI-compatible device using an HDMI cable.\n\n5.  **Thunderbolt™ 4 port**\n    The Thunderbolt™ 4 port is compatible with USB Type-C® standards, and provides a transfer rate of up to 40 Gbit/s for Thunderbolt™ 4 devices. Connect your Notebook PC to a DisplayPort, VGA, DVI, or HDMI external display and experience high-resolution display features via Intel® Thunderbolt™ technology. \n    NOTE: Transfer rate of this port may vary per model.\n\n6.  **USB 3.2 Gen 2 Type-C®/DisplayPort/Power Delivery combo port**\n    The USB 3.2 (Universal Serial Bus 3.2) Gen 2 Type-C® port provides a transfer rate of up to 10 Gbit/s and is backward compatible to USB 2.0. Use a USB Type-C® adapter to connect your Notebook PC to an external display. Connect a power source rated 20V/5A to charge the battery pack and supply power to your Notebook PC.\n    NOTE: Transfer rate of this port may vary per model.\n    WARNING! The adapter may become warm to hot while in use. Do not cover the adapter and keep it away from your body while it is connected to a power source.\n    IMPORTANT!\n    * It may take longer to fully charge your Notebook PC through a low-voltage power source.\n    * To prevent any damage, use only power sources rated 20V/5A to charge your Notebook PC with the USB Power Delivery combo port. For more information, consult an ASUS service center for assistance.\n\n7.  **Headphone/Headset/Microphone jack**\n    This port allows you to connect amplified speakers or headphones. You can also use this port to connect your headset or an external microphone."
    }
]

## Inference

Here is the text:
"""

### Count the number of tokens

Load the extracted text from the previous step.

In [2]:
file_path = "../data/extracted_text/rog_strix_gaming_notebook_pc_unscanned_file.txt"
with open(file_path, "r") as file:
    extracted_text = file.read()

len(extracted_text)

85672

Let's count the number of tokens using the token counting feature in the `genai` library.

In [3]:
from google import genai
from dotenv import load_dotenv

load_dotenv()


def count_tokens(text: str, google_client: genai.Client) -> int:
    result = google_client.models.count_tokens(model="gemini-2.5-pro", contents=text)
    return result.total_tokens  # type: ignore


google_client = genai.Client()
system_prompt_tokens = count_tokens(system_prompt, google_client)
extracted_text_tokens = count_tokens(extracted_text, google_client)
total_tokens = system_prompt_tokens + extracted_text_tokens

print(f"System prompt tokens  : {system_prompt_tokens}")
print(f"Extracted text tokens : {extracted_text_tokens}")
print(f"Total tokens          : {total_tokens}")

System prompt tokens  : 7976
Extracted text tokens : 18309
Total tokens          : 26285


### Create a Pydantic object

We use a Pydantic object to define the structure of the output we want from the LLM. This will force the LLM to return the output in a structured format that we can easily parse.

In [4]:
from pydantic import BaseModel


class TextChunk(BaseModel):
    text_chunk: str

### Use Gemini

In [5]:
response = google_client.models.generate_content(
    model="gemini-2.5-pro",
    # model="gemini-2.5-flash",
    contents=[
        system_prompt,
        extracted_text,
    ],
    config={
        "response_mime_type": "application/json",
        "response_schema": list[TextChunk],
        "max_output_tokens": 65_536,
    },
)

Because we are using a `Pydantic` object in the response schema, we need to get the parsed response from the LLM.

In [6]:
text_chunks: list[TextChunk] = response.parsed  # type: ignore
print(f"Number of text chunks: {len(text_chunks)}")

Number of text chunks: 43


Let's see count the number of tokens in the response.

In [7]:
response_text = str(response.text)
response_text_tokens = count_tokens(response_text, google_client)

print(f"Input tokens          : {total_tokens}")
print(f"Response tokens       : {response_text_tokens}")

Input tokens          : 26285
Response tokens       : 19746


That is a lot of tokens! This depends on the size of the extracted text. Just keep in mind that if you want to try doing the same thing with an open source LLM, you will need to have a lot of VRAM!

Let's inpect some chunks.

In [8]:
print(text_chunks[0].text_chunk)

E25294
REVISED EDITION V5 / DECEMBER 2024

ROG STRIX
GAMING NOTEBOOK PC


In [9]:
print(text_chunks[10].text_chunk)

Battery safety information

Battery protection
*   Frequently charging a battery under high voltage may shorten the battery life. To protect the battery, once the battery is fully charged, the system may stop charging when the battery power is between 90% and 100%.
    NOTE: The level at which the battery resumes charging is usually set between 90% and 99%. The actual value may vary by model.
*   Charging or storing a battery in high ambient temperatures may result in permanent damage to the battery power capacity and rapidly shorten battery life. If the battery temperature is too high, the system may reduce the battery power charging capacity or even stop charging to protect the battery.
*   Battery power capacity may drop even when your device is powered off and disconnected from the AC power. This is normal as the system still draws a small amount of power from the battery.

Standard battery care
*   If you are not using your device for a long period of time, ensure to charge the ba

In [10]:
print(text_chunks[20].text_chunk)

Windows keys
There are two special Windows keys on your Notebook PC’s keyboard used as below:
-   Windows logo key: Launches the Start menu
-   Context menu key or FN + CTRL: Displays the drop-down menu


The model will try its best to follow the system prompt and divide the text into chunks that each focus on a single topic.

You have full control on what to do with these chunks. If you think that two chunks can be merged into one, you can do that. If you think that some chunks are not useful, you can discard them. The important thing is that each chunk should focus on a single topic.

### Save the chunks

Convert the list of Pydantic objects into a JSON object to make it easier for you to modify or inspect the chunks later on.


In [11]:
text_chunks_dict = [
    {"id": i, "text_chunk": chunk.text_chunk}
    for i, chunk in enumerate(text_chunks)
]
text_chunks_dict[0].keys()

dict_keys(['id', 'text_chunk'])

In [13]:
import os
import json

chunks_directory = "../data/chunks/"
os.makedirs(chunks_directory, exist_ok=True)

file_name = file_path.split("/")[-1].replace(".txt", "_chunks.json")
output_file_path = os.path.join(chunks_directory, file_name)

with open(output_file_path, "w") as f:
    json.dump(text_chunks_dict, f, indent=2)

Now, you can modify the JSON file as you like!